In [173]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt

# In[10]:


def World_Bank_Modification(data):
    indicator = data['Indicator Name'].unique()[0]
    data = data.drop(columns=['Indicator Name', 'Country Code', 'Indicator Code'])
    data = data.set_index('Country Name').stack().reset_index().rename(columns={'Country Name':'country', 'level_1':'year',0:indicator})
    data.year = pd.to_numeric(data.year, errors='coerce')
    data.year = pd.to_datetime(data['year'],format='%Y')
    #国名修正
    data.country = data.country.replace('Hong Kong SAR, China', 'Hong Kong')
    data.country = data.country.replace('Korea, Rep.', 'South Korea')
    return data[data.country.isin(country)].set_index(['country','year'])



# In[11]:


path =['gross_value_added.csv','industry_value_added.csv', 'import.csv','export.csv','gdp_deflator.csv']


# In[12]:


#######ここから
##Gross だけ初めから入れておく
iterables = [country, pd.to_datetime(np.arange(1960,1997),format='%Y')]
index = pd.MultiIndex.from_product(iterables, names=['country', 'year'])
df = pd.DataFrame({'value':np.nan},index=index)


##他を読み込んで統合
for file in path:
    a = pd.read_csv(file,skiprows=4, encoding='cp932')
    df = pd.merge(df, World_Bank_Modification(a),left_index=True, right_index=True, how='left')
    
##最初に入れたNANをDrop
df = df.drop(columns='value')

#column名変更
df.columns = ['GDP', 'IND', 'M', 'X','DEFLATOR'] 


In [174]:
countries = ['Singapore', 'Indonesia', 'India', 'Malaysia', 'Philippines', 'Thailand', 'Taiwan','Japan']
# countries = ['Singapore','India','Philippines']

In [175]:
df = df.reset_index()

In [176]:
# df

In [177]:
df = df[df.country.isin(countries)]

In [161]:
# df.unstack(level=0).head().loc[:]['DEFLATOR']

In [162]:
##デフレータを１９８７年に統一する方法はよくわからんからあとでやる。

In [163]:
# ##deflator_calculation
# d_c = pd.DataFrame({'DEFLATOR':df.DEFLATOR},index=df.index).unstack(level=0)
# d_c = d_c.reset_index()
# # d_c.loc[:]['DEFLATOR'] = d_c.loc[:]['DEFLATOR'] / d_c.loc[d_c.year=='1987-01-01'][:]
# d_c.loc[:]['DEFLATOR'] / d_c.loc[d_c.year=='1987-01-01'][:].DEFLATOR

# plt.plot(d_c.year,d_c.DEFLATOR)

In [164]:
##データ整形
#1987年で調整
def constantPrice(df):
    df.loc[:]['M'] = df.loc[:]['M'] / df.loc[df.year=='1987-01-01']['M']
    df.loc[:]['X'] = df.loc[:]['X'] / df.loc[df.year=='1987-01-01']['X']
    df.loc[:]['DEFLATOR'] = df.loc[:]['DEFLATOR'] / df.loc[df.year=='1987-01-01']['DEFLATOR']
    df.loc[:]['LP'] = df.loc[:]['LP'] / df.loc[:]['DEFLATOR']
    
    return df

In [179]:
df[df.country=='Japan'].to_csv('Japan_data.csv',index=None)

In [181]:
# df

In [182]:

# #Log Form
# #Labor Productivityはインフレ率を考慮
# df.loc[:]['LP'] = np.log(df.loc[:]['LP']/df.loc[:]['DEFLATOR'])
# df.loc[:]['M'] = np.log(df.loc[:]['M'])
# df.loc[:]['X'] = np.log(df.loc[:]['X'])

# df = df.drop(['DEFLATOR'],axis=1)

In [184]:
# df